# Árvores II - Tarefa 2

### 1. Carregar as bases

Vamos carregar as bases lidas na tarefa passada. Se você salvou essas bases em arquivo texto, basta fazer a leitura com o comando ```pd.read_csv``` das seguintes bases:

- X_train
- Y_train
- X_test
- Y_test

Não se esqueça de considerar a leitura dos índices dos arquivos no ```read_csv()```!

In [112]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
#from sklearn.metrics import plot_confusion_matrix

In [113]:
def load_data(fn_subject, fn_x, fn_export):

  df_features = pd.read_csv('features.txt', delimiter='\s+', header=None)
  df_subject = pd.read_csv(fn_subject, delimiter='\s+', header=None)
  df_x = pd.read_csv(fn_x, delimiter='\s+', header=None)

  df_x.columns = df_features[1]
#  df_x = df_x[['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z']]
  df_x['subject_id'] = df_subject[0]

  df_x = df_x.reset_index()
  df_x = df_x.set_index(['index', 'subject_id'])
#  df_x.to_csv(fn_export)

  return df_x

df_x_train_base = load_data('subject_train.txt', 'X_train.txt', 'df_x_train.csv')
df_y_train_base = pd.read_csv('y_train.txt', delimiter='\s+', header=None)

#df_x_test = load_data('subject_test.txt', 'X_test.txt', 'df_x_test.csv')
#df_y_test = pd.read_csv('y_test.txt', delimiter='\s+', header=None)

### 2. Divisão da base em Treino, Validação e Teste

A base já se encontra dividida em Treino e Validação. O que vamos fazer então é extrair uma base de Validação da base de Treino.

Extraia 25% da base de treino como base de validação.

In [114]:
df_x_train, df_x_valid, df_y_train, df_y_valid = train_test_split(df_x_train_base, df_y_train_base, test_size=.25, random_state=1729)

### 3. Melhores 3 variáveis

Rode uma árvore com profundidade máxima igual a 4 para prever a atividade humana com todas as variáveis.
Observe a importância das variáveis e considere as 3 variáveis com maior importância para os próximos passos.
Dica: utilize o atributo ```clf.feature_importances_``` da árvore treinada.

In [115]:
clf = DecisionTreeClassifier(random_state=100, max_depth=4).fit(df_x_train, df_y_train)

In [116]:
#df_features = pd.DataFrame([clf.feature_importances_], columns=df_x_train.columns).T
#precisei usar os índices da coluna pq tem feature com nome repetido

df_features = pd.DataFrame({'feature_importance': clf.feature_importances_})
df_best_features = df_features['feature_importance'].sort_values(ascending=False)[0:3]
df_best_features.index

Int64Index([40, 393, 559], dtype='int64')

In [117]:
df_x_train = df_x_train.iloc[:,df_best_features.index]
df_x_train

,1,tGravityAcc-mean()-X,"fBodyAccJerk-bandsEnergy()-1,24","angle(Y,gravityMean)"
index,subject_id,,,
3229,16,0.878431,-0.888136,-0.008996
6988,30,0.962831,-0.999351,0.210314
5189,25,0.883538,-0.889242,0.163278
6263,28,0.954383,-0.999980,0.247250
888,5,0.978183,-0.999280,0.132197
...,...,...,...,...
3605,19,0.939822,-0.998697,0.251008
5669,26,0.935296,-0.759234,0.252466
1267,6,0.909636,-0.253555,0.221396


### 4. Construa uma árvore com as 3 melhores variáveis

Utilizando as três variáveis encontradas acima, construa uma árvore de decisão. Encontre o melhor ```ccp_alpha``` utilizando a base de validação, conforme a estrutura que vimos em aula.

In [118]:
%%time
clf_ccp_alpha=DecisionTreeClassifier(random_state=999)
path = clf_ccp_alpha.cost_complexity_pruning_path(df_x_train, df_y_train)
ccp_alphas = path.ccp_alphas

CPU times: user 31.3 ms, sys: 0 ns, total: 31.3 ms
Wall time: 31.7 ms


In [119]:
clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=2360873, ccp_alpha=ccp_alpha).fit(df_x_train, df_y_train)
    clfs.append(clf)

train_scores = [clf.score(df_x_train, df_y_train) for clf in clfs]

df_x_valid = df_x_valid.iloc[:,df_best_features.index]
valid_scores  = [clf.score(df_x_valid, df_y_valid) for clf in clfs]

ind_melhor_arvore_valid = len(valid_scores) - valid_scores[::-1].index(max(valid_scores)) - 1
melhor_arvore = clfs[ind_melhor_arvore_valid]
print(f'Acurácia da melhor árvore na base de validação: {valid_scores[ind_melhor_arvore_valid]*100:.2f}')

Acurácia da melhor árvore na base de validação: 86.13


### 5. Avaliação do modelo

Avalie a árvore encontrada no item anterior na base de testes.

In [120]:
df_x_test = df_x_test.iloc[:,df_best_features.index]
test_scores  = [clf.score(df_x_test, df_y_test) for clf in clfs]

ind_melhor_arvore_test = len(test_scores) - test_scores[::-1].index(max(test_scores)) - 1
melhor_arvore = clfs[ind_melhor_arvore_test]
print(f'Acurácia da melhor árvore na base de testes: {test_scores[ind_melhor_arvore_test]*100:.2f}')

Acurácia da melhor árvore na base de testes: 71.90
